# Data Import

In [1]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
from transformers import TFBertModel, BertTokenizer, DistilBertTokenizer, TFDistilBertModel
from sklearn.model_selection import train_test_split
from sklearn import metrics


2023-05-02 03:34:38.620465: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-02 03:34:38.668804: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-02 03:34:38.669420: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 03:34:39.284255: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
data = pd.read_json('sarcasm.json',lines=True)
labels = data.is_sarcastic.values
sentences = data.headline.values
data.head()


,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


# DistillBert

In [4]:
train_sents,test_sents, train_labels, test_labels  = train_test_split(sentences,labels,test_size=0.1)

In [5]:
PRE_TRAINED_MODEL_NAME = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME,do_lower_case = True)

def encoder_dis(sentences):
  ids = []
  for sentence in sentences:
    encoding = tokenizer.encode_plus(
    sentence,
    max_length=16,
    truncation = True,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=False)
    ids.append(encoding['input_ids'])
  return ids

train_ids_dis = encoder_dis(train_sents)
test_ids_dis = encoder_dis(test_sents) 

/opt/tljh/user/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [6]:
train_ids_dis = tf.convert_to_tensor(train_ids_dis)
test_ids_dis = tf.convert_to_tensor(test_ids_dis)
test_labels_dis = tf.convert_to_tensor(test_labels)
train_labels_dis = tf.convert_to_tensor(train_labels)

2023-05-02 03:34:50.734482: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
distillbert_encoder = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
input_word_ids = tf.keras.Input(shape=(16,), dtype=tf.int32, name="input_word_ids")  
embedding = distillbert_encoder([input_word_ids])
dense = tf.keras.layers.Lambda(lambda seq: seq[:, 0, :])(embedding[0])
dense = tf.keras.layers.Dense(128, activation='relu')(dense)
dense = tf.keras.layers.Dropout(0.2)(dense)   
output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)    

model_dis = tf.keras.Model(inputs=[input_word_ids], outputs=output)  

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'vocab_transform', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [8]:
model_dis.compile(tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
trained_dis = model_dis.fit(x = train_ids_dis, y = train_labels_dis, epochs = 3, verbose = 1, batch_size = 32, validation_split=0.1)

Epoch 1/3
725/725 [==============================] - 405s 547ms/step - loss: 0.4301 - accuracy: 0.7910 - val_loss: 0.3279 - val_accuracy: 0.8579
Epoch 2/3
725/725 [==============================] - 392s 541ms/step - loss: 0.2840 - accuracy: 0.8796 - val_loss: 0.3121 - val_accuracy: 0.8700
Epoch 3/3
725/725 [==============================] - 392s 541ms/step - loss: 0.1996 - accuracy: 0.9188 - val_loss: 0.3016 - val_accuracy: 0.8804


In-distribution Test

In [10]:
predictions_indis = [1 * (x[0]>=0.5) for x in model_dis.predict(test_ids_dis)]
accuracy = metrics.accuracy_score(test_labels_dis, predictions_indis)
print(accuracy)

90/90 [==============================] - 12s 123ms/step
0.8784067085953878


In [11]:
#generate ids for reddit test data
test_data1 = pd.read_csv('clean_reddit.csv')
test_labels1 = test_data1.label.values
test_sentences1 = test_data1.sent.values
test_data1.head()


test_ids1 = encoder_dis(test_sentences1)
test_ids1 = tf.convert_to_tensor(test_ids1)
test_labels_dis1 = tf.convert_to_tensor(test_labels1)
# print("first 5 test_ids1: ", test_ids1[:5])
# print("first 5 test_labels_dis1: ", test_labels_dis1[:5])


#generate ids for tweet test data
test_data2 = pd.read_csv('clean_tweet.csv')
test_labels2 = test_data2.label.values
test_sentences2 = test_data2.sent.values
test_data2.head()


test_ids2 = encoder_dis(test_sentences2)
test_ids2 = tf.convert_to_tensor(test_ids2)
test_labels_dis2 = tf.convert_to_tensor(test_labels2)
# print("first 5 test_ids2: ", test_ids2[:5])
# print("first 5 test_labels_dis2: ", test_labels_dis2[:5])




/opt/tljh/user/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/opt/tljh/user/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal 

Out-of-distribution Tests on Reddits and Tweets

In [12]:
predictions_reddit = [1 * (x[0]>=0.5) for x in model_dis.predict(test_ids1)]
accuracy = metrics.accuracy_score(test_labels_dis1, predictions_reddit)
print(accuracy)







42/42 [==============================] - 5s 121ms/step
0.48760330578512395


In [13]:
predictions_tweets = [1 * (x[0]>=0.5) for x in model_dis.predict(test_ids2)]
accuracy = metrics.accuracy_score(test_labels_dis2, predictions_tweets)
print(accuracy)

46/46 [==============================] - 6s 120ms/step
0.5152143845089903


For Baseline Testing (Untrained model)

In [14]:
distillbert_encoder = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
input_word_ids = tf.keras.Input(shape=(16,), dtype=tf.int32, name="input_word_ids")  
embedding = distillbert_encoder([input_word_ids])
dense = tf.keras.layers.Lambda(lambda seq: seq[:, 0, :])(embedding[0])
dense = tf.keras.layers.Dense(128, activation='relu')(dense)
dense = tf.keras.layers.Dropout(0.2)(dense)   
output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)    

model_base = tf.keras.Model(inputs=[input_word_ids], outputs=output)  
model_base.compile(tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'vocab_transform', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [15]:
predictions_reddit_base = [1 * (x[0]>=0.5) for x in model_base.predict(test_ids1)]
accuracy = metrics.accuracy_score(test_labels_dis1, predictions_reddit_base)
print(accuracy)

42/42 [==============================] - 6s 122ms/step
0.5093914350112697


In [16]:
predictions_tweet_base = [1 * (x[0]>=0.5) for x in model_base.predict(test_ids2)]
accuracy = metrics.accuracy_score(test_labels_dis2, predictions_tweet_base)
print(accuracy)

46/46 [==============================] - 6s 121ms/step
0.4778699861687414


In [17]:
predictions_news = [1 * (x[0]>=0.5) for x in model_base.predict(test_ids_dis)]
accuracy = metrics.accuracy_score(test_labels_dis, predictions_news)
print(accuracy)

90/90 [==============================] - 11s 122ms/step
0.4744933612858141
